# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]
#type(filepath)

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARULZCI1241B9C8611,NaN,,NaN,Luna Orbit Project,335.51628,1,SOSWKAV12AB018FC91,Midnight Star,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
# Extract columns from the DataFrame to add to our "songs" table
#song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()[0]
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
# Select the first record in the dataframe
song_data= song_data.head(1)
# Set to list
song_data = song_data.values.tolist()
song_data = song_data[0]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
# Extract columns from the DataFrame to add to our "artist" table
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
# Select the first record in the dataframe
artist_data= artist_data.head(1)
# Set to list
artist_data = artist_data.values.tolist()
artist_data = artist_data[0]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("data/log_data")

In [12]:
filepath = log_files[0]

In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,741,None,307,1542760054796,None,
1,None,Logged In,Theodore,M,1,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,741,None,200,1542760086796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
2,None,Logged Out,None,None,0,None,NaN,paid,None,GET,Home,NaN,774,None,200,1542761399796,None,
3,None,Logged Out,None,None,1,None,NaN,paid,None,GET,Home,NaN,774,None,200,1542761485796,None,
4,None,Logged Out,None,None,2,None,NaN,paid,None,PUT,Login,NaN,774,None,307,1542761486796,None,


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df['page']=="NextSong"]
#df.head()
#df.shape

In [15]:
t = pd.to_datetime(df['ts'], unit='ms')
#type(t)
#t.head()

In [16]:
th = (t.dt.time, t.dt.hour, t.dt.day,  t.dt.week, t.dt.month,  t.dt.year, t.dt.weekday)
th = list(th)
#type(th)

In [17]:
time_data = (th)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [18]:
time_df = pd.DataFrame(time_data, index = column_labels)
time_df = time_df.T
time_df.head()
#time_df.shape

,start_time,hour,day,week,month,year,weekday
6,00:57:58.796000,0,21,47,11,2018,2
7,00:58:41.796000,0,21,47,11,2018,2
8,01:02:05.796000,1,21,47,11,2018,2
9,01:03:13.796000,1,21,47,11,2018,2
11,01:05:29.796000,1,21,47,11,2018,2


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
#user_df

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
        
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    # insert songplay record
    #songplay_data = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [24]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.